# Trimming and Filtering a FASTQ
<!---
** Add Intro Text***
--->

## Shell Variables
Assign the variables in this notebook.

In [ ]:
# We used these in the last notebook
CUROUT=$HOME/work/scratch/2015_output
DEMUX=$CUROUT/demux_fastqs

# The following are new for this notebook
TRIMMED=$CUROUT/trimmed_fastqs
INFO=$HOME/work/myinfo

## Making New Directories
Make the directories that are new in this notebook

In [ ]:
mkdir -p $TRIMMED
mkdir -p $INFO

Now let's check to be sure that worked.  We will run `ls` and check that these directories now exist in the `$CUROUT` directory.

In [ ]:
ls $CUROUT

# Trimming and Filtering
Now we get into some actual preprocessing.  We will use `fastq-mcf` to trim adapter from our reads and do some quality filtering.  We need to trim adapter, because if a fragment is short enough, we will sequence all the way through the fragment and into the adapter.  Obviously the adapter sequence in not found in the genome, and can keep the read from aligning properly.  To do the trimming, we need to generate an adapter file.

## Making an adapter file
The first step is to get the adapter sequence.  For our test file, the index is 16 (barcode CCGTCC), this is from NEBNext Multiplex Oligos for Illumina (Index Primers Set 2).  So let's look in the manual: 

https://www.neb.com/~/media/Catalog/All-Products/6B6FC6C03B274E7FA0FDBF13015AB194/Datacards%20or%20Manuals/manualE7500.pdf

We will get the sequence for the universal primer and the index primer.  Now we need to make the adapter file; this needs to be in FASTA format.

1. click on the jupyter "File" menu, and select "Open".  
2. When the the new browser window/tab opens, click on the "Files" tab if it is not already active.
3. Click on the "home" symbol to go to the top level directory, then click on "myinfo"
4. In the "New" menu select "Text File".
5. In this text file, type ">adapter_3prime", this is the name for the adapter sequence, the ">" is part of the FASTA format indicating that it is the sequence name.
6. Next copy the part of the 3' common part of the adapter, the part that is to the right of the barcode sequence, and paste it on the line after the name.
7. We also want to include the reverse complement of the adapter, in case that is what will actually be sequenced.  The easiest way to do that is to use https://www.bioinformatics.org/sms/rev_comp.html to generate the reverse complement, then name it something like "adapter_3prime_RC"
8. Now clean up by making sure that . . .
    1. The sequence is on its own line
    2. The sequence has a name on the line before it
    3. The name is preceded by a ">"
    4. All spaces and non-sequence characters have been removed
9. We need to repeat steps 5-8 for the **universal** adapter sequence that is in the same PDF.
10. Click on "untitled.txt" to change the file name to "neb_adapters.fasta"
11. Save the file.


## fastq-mcf
You can run `fastq-mcf -h` to get details about running fastq-mcf.  We will adjust run parameters, because some of the defaults set a low bar (even the author acknowleges this).

In [ ]:
# the "| cat" is a hack that prevents problems with jupyter
fastq-mcf -h | cat

### Running fastq-mcf
1. neb_adapters.fasta : the adapter file
2. r1.8A_pilot.fq.gz : the FASTQ with the data (fastq-mcf, like most NGS analysis software, detects gzipped files and automatically decompresses on the fly)
3. -q 20 : if a read has any bases with quality score lower than this, trim them and anything 3' of that base
4. -x 0.5 : if this percentage (or higher) of the reads have an "N" in a given position, trim all reads to that position
5. -o r1.test.trim.fastq.gz : output file (the .gz ending tells fastq-mcf to compress the output file)

In [ ]:
fastq-mcf $INFO/neb_adapters.fasta \
    $DEMUX/r1.8A_pilot.fq.gz \
    -q 20 -x 0.5 \
    -o $TRIMMED/r1.8A_pilot.trim.fastq.gz

Notice that it was the reverse complement of the adapter that was actually found, so its a good thing that we actually included it.

at this point we could run fastqc on the output of fastq-mcf to see if statistics have improved, but we will skip that for now.

In [ ]:
ls $TRIMMED